In [24]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [25]:
QB = pd.read_csv('skilllevel3.csv')

In [26]:
QB['RunningSkill'] = ' '

In [27]:
QB

Year  Fantasy_Points  PassingYards  PassingTDs  Ints  RushAttempts  \
0    2019-01-01              16           225           0     1             5   
1    2020-01-01               0            20           0     0             0   
2    2023-01-01               0            19           0     0             0   
3    2019-01-01             322          4002          26     4            46   
4    2020-01-01             469          4299          48     5            38   
..          ...             ...           ...         ...   ...           ...   
393  2019-01-01               0           228           0     4             7   
394  2023-01-01             115          1808           8     4            25   
395  2021-01-01             163          2334           9    11            29   
396  2022-01-01             105          1688           6     7            28   
397  2023-01-01             132          2271           8     7            36   

     RushYards  Fumbles  RushTDs PassingSkill RunningSkill  
0           39        0        1       Backup               
1            0        0        0       Backup               
2            0        0        0       Backup               
3          183        4        1  Really Good               
4          149        2        3   MVP Passer               
..         ...      ...      ...          ...          ...  
393         22        1        0       Backup               
394         57        4        1       Backup               
395        185        1        4       Backup               
396        102        1        1       Backup               
397        211        7        0       Backup               

[398 rows x 11 columns]

In [28]:
QB['Year'] = pd.to_datetime(QB['Year'])

In [29]:
QB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Year            398 non-null    datetime64[ns]
 1   Fantasy_Points  398 non-null    int64         
 2   PassingYards    398 non-null    int64         
 3   PassingTDs      398 non-null    int64         
 4   Ints            398 non-null    int64         
 5   RushAttempts    398 non-null    int64         
 6   RushYards       398 non-null    int64         
 7   Fumbles         398 non-null    int64         
 8   RushTDs         398 non-null    int64         
 9   PassingSkill    398 non-null    object        
 10  RunningSkill    398 non-null    object        
dtypes: datetime64[ns](1), int64(8), object(2)
memory usage: 34.3+ KB


In [30]:
QB.set_index('Year', inplace=True)

In [31]:
QB.loc[(QB['RushYards'] >= 300) & (QB['RushTDs'] >= 5), 'RunningSkill'] = 'Dual Threat'
QB.loc[(QB['RushYards'] >= 400) & (QB['RushTDs'] <= 5), 'RunningSkill'] = 'Dual Threat'
QB.loc[(QB['RushYards'] < 300) & (QB['RushTDs'] >= 5), 'RunningSkill'] = 'RedZone Threat'
QB.loc[(QB['RushYards'] < 400) & (QB['RushTDs'] < 5), 'RunningSkill'] = 'RegularQB'

In [32]:
QB

Fantasy_Points  PassingYards  PassingTDs  Ints  RushAttempts  \
Year                                                                       
2019-01-01              16           225           0     1             5   
2020-01-01               0            20           0     0             0   
2023-01-01               0            19           0     0             0   
2019-01-01             322          4002          26     4            46   
2020-01-01             469          4299          48     5            38   
...                    ...           ...         ...   ...           ...   
2019-01-01               0           228           0     4             7   
2023-01-01             115          1808           8     4            25   
2021-01-01             163          2334           9    11            29   
2022-01-01             105          1688           6     7            28   
2023-01-01             132          2271           8     7            36   

            RushYards  Fumbles  RushTDs PassingSkill RunningSkill  
Year                                                               
2019-01-01         39        0        1       Backup    RegularQB  
2020-01-01          0        0        0       Backup    RegularQB  
2023-01-01          0        0        0       Backup    RegularQB  
2019-01-01        183        4        1  Really Good    RegularQB  
2020-01-01        149        2        3   MVP Passer    RegularQB  
...               ...      ...      ...          ...          ...  
2019-01-01         22        1        0       Backup    RegularQB  
2023-01-01         57        4        1       Backup    RegularQB  
2021-01-01        185        1        4       Backup    RegularQB  
2022-01-01        102        1        1       Backup    RegularQB  
2023-01-01        211        7        0       Backup    RegularQB  

[398 rows x 10 columns]

In [33]:
filter_values = ['Dual Threat','RedZone Threat']
RunningQB = QB[QB['RunningSkill'].isin(filter_values)]


In [34]:

X = QB.drop(['Fantasy_Points'], axis=1)
y = QB['Fantasy_Points']


numeric_features = ['PassingTDs']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
     
    ]
)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 614.6019272063073
R-squared: 0.9665676364788525


In [35]:


X = RunningQB.drop(['Fantasy_Points'], axis=1)
y = RunningQB['Fantasy_Points']



numeric_features = ['PassingTDs']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
 
    ]
)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 1233.48823449286
R-squared: 0.703583410487137


In [20]:
len(RunningQB)

41

In [21]:
len(QB)

398